In [ ]:
import os
import re
import shutil
from datetime import datetime
from collections import defaultdict

import numpy as np
import cv2
import torch
from bs4 import BeautifulSoup, Tag
from PIL import Image, ImageChops, ImageDraw
from realesrgan.utils import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

from api_handler import MistralAPI
from local_model import LocalModel

In [285]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [286]:
def read_config_paths(path_file="config_path.txt"):
    path_config = {}
    
    if not os.path.exists(path_file):
        raise FileNotFoundError(f"Config file not found: {path_file}")

    with open(path_file, "r") as f:
        for line in f:
            if "=" in line:
                key, value = line.strip().split("=", 1)
                path_config[key.strip()] = value.strip()

    try:
        variant = path_config["variant"]
        report_path = path_config["report_path_template"].format(variant=variant)
        image_dir = path_config["image_dir_template"].format(variant=variant)
        output_dir = path_config["output_dir"]
        relative_image_path = path_config["relative_image_path_template"].format(variant=variant)
    except KeyError as e:
        raise KeyError(f"Missing required config key: {e}")

    # Ensure output folder exists
    os.makedirs(output_dir, exist_ok=True)
    
    return {
        "variant": variant,
        "report_path": report_path,
        "image_dir": image_dir,
        "output_dir": output_dir,
        "relative_image_path_for_html": relative_image_path
    }

In [287]:
# Load config paths
paths = read_config_paths("config_path.txt")

In [288]:
# === Format variant name ===
def find_variant_display_name():    
    variant_display_name = paths["variant"].replace("_Regression_Tests_globalTestReport", "")
    variant_display_name = variant_display_name.replace("_", " ")

    if "Legacy" in paths["variant"]:
        variant_display_name += " Legacy"

    return variant_display_name

In [289]:
def get_unique_path(base_path):
    """Return a unique path by appending (1), (2), etc. if needed."""
    if not os.path.exists(base_path):
        return base_path

    base, ext = os.path.splitext(base_path)
    counter = 1
    while True:
        new_path = f"{base} ({counter}){ext}"
        if not os.path.exists(new_path):
            return new_path
        counter += 1

In [290]:
# Define paths and create directories
image_dir = paths["image_dir"]
relative_image_path = paths["relative_image_path_for_html"]

variant_display_name = find_variant_display_name()
output_dir = paths["output_dir"]
output_html = get_unique_path(os.path.join(output_dir, f"aiAnalysisV2Report_{variant_display_name}.html"))
output_img_dir = get_unique_path(os.path.join(output_dir, "img"))
os.makedirs(output_img_dir, exist_ok=True)

In [291]:
def compute_scale_mismatch(img1, img2):
    w1, h1 = img1.size
    w2, h2 = img2.size
    w_diff = abs(w1 - w2) / max(w1, w2)
    h_diff = abs(h1 - h2) / max(h1, h2)
    return round((w_diff + h_diff) / 2 * 100, 2)

In [292]:
# Initialize RRDBNet and RealESRGAN model only once
model = RRDBNet(
    num_in_ch=3,
    num_out_ch=3,
    num_feat=64,
    num_block=23,
    num_grow_ch=32,
    scale=4
)

upsampler = RealESRGANer(
    scale=4,
    model_path='./pkg_instl_files/Real-ESRGAN-master/realesrgan/weights/RealESRGAN_x4plus.pth',
    model=model,
    tile=0,
    tile_pad=10,
    pre_pad=0,
    half=False,
)

def realesrgan_enlarger_enhancer(image):
    """
    Enhance and upscale the input PIL image using Real-ESRGAN.

    Args:
        image (PIL.Image): Input low-resolution image.

    Returns:
        PIL.Image: Enhanced and upscaled image.
    """
    img_np = np.array(image)
    sr_image_np, _ = upsampler.enhance(img_np)
    sr_image_clipped = np.clip(sr_image_np, 0, 255).astype("uint8")
    return Image.fromarray(sr_image_clipped).convert("RGB")


In [293]:
def classify_diff_distribution(binary_mask, min_cluster_size, global_threshold, max_clusters_allowed):
    """
    Classify differences as 'global' or 'localized' based on:
      1. Bounding-box coverage of the largest cluster
      2. Number of non-overlapping clusters
    
    Args:
        binary_mask: np.uint8 mask where 255 = red diff, 0 = no diff
        min_cluster_size: ignore blobs smaller than this (pixel area)
        global_threshold: fraction of image area above which the largest cluster
                          is considered a global shift
        max_clusters_allowed: number of clusters allowed before treating as global
    Returns:
        classification: "global_shift", "localized", or "no_diff"
        cluster_count: number of valid clusters
    """

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
        binary_mask, connectivity=8
    )

    h_img, w_img = binary_mask.shape
    img_area = h_img * w_img

    bbox_areas = []
    cluster_count = 0
    for i in range(1, num_labels):  # skip background (0)
        area = stats[i, cv2.CC_STAT_AREA]
        if area < min_cluster_size:
            continue
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        bbox_area = w * h
        bbox_areas.append(bbox_area)
        cluster_count += 1

    if not bbox_areas:
        return "no_diff", 0

    max_bbox_fraction = max(bbox_areas) / img_area

    # Rule 1: bounding box area fraction
    if max_bbox_fraction >= global_threshold:
        return "global_shift", cluster_count

    # Rule 2: too many clusters
    if cluster_count > max_clusters_allowed:
        return "global_shift", cluster_count

    return "localized", cluster_count


In [294]:
def highlight_differences(img1, img2, highlight_color=(255, 0, 0), threshold=30):
    """
    Overlay img1 and img2 in grayscale and highlight perceptible differences
    in the specified highlight_color. Ignores small differences below threshold.
    
    Args:
        img1, img2: PIL.Image objects (RGB or grayscale)
        highlight_color: Tuple of RGB values to highlight differences
        threshold: Int from 0–255. Lower = more sensitive, higher = less sensitive.
    """

    # Convert both images to grayscale
    img1_gray = img1.convert("L")
    img2_gray = img2.convert("L")

    # Convert to numpy arrays for precise diff and thresholding
    arr1 = np.array(img1_gray).astype(np.int16)
    arr2 = np.array(img2_gray).astype(np.int16)

    # Compute absolute pixel-wise difference
    diff = np.abs(arr1 - arr2)

    # Apply threshold to ignore small changes
    diff_mask = (diff > threshold).astype(np.uint8) * 255  # binary mask

    # Boolean: check if any pixel is different
    has_diff = np.any(diff_mask)
    
    # Convert diff_mask to PIL image
    diff_mask_img = Image.fromarray(diff_mask, mode="L")

    # Convert one of the grayscales to RGB for base visualization
    base = img1_gray.convert("RGB")

    # Create a solid highlight image
    highlight = Image.new("RGB", base.size, highlight_color)

    # Composite the highlight where differences exist
    result = Image.composite(highlight, base, diff_mask_img)

    # -----------------------------------
    # Classifying Difference Distribution
    # -----------------------------------
    
    result_type, cluster_count = classify_diff_distribution(diff_mask, min_cluster_size=5, global_threshold=0.25, max_clusters_allowed=50)

    
    # ----------------------
    # Calculate R/W ratios
    # ----------------------

    diff_mask_bin = (diff_mask > 0).astype(np.uint8)  # binary 1 = red highlight region

    # Count red pixels (where diff > threshold)
    red_pixels = np.sum(diff_mask_bin)

    # Reference white pixels (in img1)
    ref_white = np.sum(np.array(img1_gray) > 127)  # threshold grayscale
    # To-check white pixels (in img2)
    check_white = np.sum(np.array(img2_gray) > 127)

    # Avoid division by zero
    # rw_score_ref = red_pixels / ref_white if ref_white > 0 else red_pixels
    # rw_score_check = red_pixels / check_white if check_white > 0 else red_pixels

    return (red_pixels, result, has_diff, ref_white, check_white, result_type, cluster_count)


In [295]:
# def detect_element_shifts(img1_path, img2_path, visualize=False, min_match_count=10):
#     img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)  # reference
#     img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)  # to check

#     # Step 1: Detect ORB keypoints and descriptors
#     orb = cv2.ORB_create(1000)
#     kp1, des1 = orb.detectAndCompute(img1, None)
#     kp2, des2 = orb.detectAndCompute(img2, None)

#     # Step 2: Match descriptors using FLANN
#     index_params = dict(algorithm=6,  # FLANN_INDEX_LSH
#                         table_number=6,
#                         key_size=12,
#                         multi_probe_level=1)
#     search_params = dict(checks=50)
#     flann = cv2.FlannBasedMatcher(index_params, search_params)

#     # ORB needs uint8 descriptors
#     if des1 is None or des2 is None:
#         print("No keypoints found in one of the images.")
#         return {}

#     matches = flann.knnMatch(des1, des2, k=2)

#     # Step 3: Lowe’s ratio test to filter good matches
#     good_matches = []
#     for m_n in matches:
#         if len(m_n) < 2:
#             continue  # skip invalid match
#         m, n = m_n
#         if m.distance < 0.7 * n.distance:
#             good_matches.append(m)

#     print(f"Good Matches: {len(good_matches)}")

#     if len(good_matches) < min_match_count:
#         print("Not enough good matches to analyze shift.")
#         return {}

#     # Step 4: Analyze shift in positions
#     shifts = []
#     for match in good_matches:
#         pt1 = kp1[match.queryIdx].pt  # from img1
#         pt2 = kp2[match.trainIdx].pt  # from img2
#         dx = pt2[0] - pt1[0]
#         dy = pt2[1] - pt1[1]
#         shifts.append((dx, dy))

#     shifts = np.array(shifts)
#     avg_shift = np.mean(shifts, axis=0)
#     max_shift = np.max(np.abs(shifts), axis=0)
#     std_shift = np.std(shifts, axis=0)

#     print(f"Check Image: {os.path.basename(img1_path)}")
#     print(f"Average Shift: dx={avg_shift[0]:.2f}, dy={avg_shift[1]:.2f}")
#     print(f"Maximum Shift: dx={max_shift[0]:.2f}, dy={max_shift[1]:.2f}")
#     print(f"Standard Deviation of Shift: dx={std_shift[0]:.2f}, dy={std_shift[1]:.2f}")

#     # if visualize:
#     #     img_match = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#     #     plt.figure(figsize=(15, 6))
#     #     plt.imshow(img_match)
#     #     plt.title("ORB + FLANN Matches (Yellow lines show matched keypoints)")
#     #     plt.axis('off')
#     #     plt.show()

#     return {
#         "average_shift": avg_shift.tolist(),
#         "max_shift": max_shift.tolist(),
#         "std_shift": std_shift.tolist(),
#         "num_good_matches": int(len(good_matches))
#     }


In [296]:
def AI_analyzer(image1: Image.Image, image2: Image.Image):   

    # Check for API key
    api_key_path = "api_key.txt"

    try:
        with open(api_key_path, "r") as f:
            api_key = f.read().strip()
    except FileNotFoundError:
        print("API key file not found. Using local model...")
        api_key = None

    if api_key:
        print("Using Mistral API...")
        mistral = MistralAPI(api_key)
        response = mistral.compare_images(image1, image2)
    else:
        print("No API key found. Using local model...")
        local = LocalModel()
        response = local.compare_images(image1, image2)
        
    print("\nDifference:\n", response)

    # os.makedirs("output", exist_ok=True)
    # with open("output/difference.txt", "w", encoding="utf-8") as f:
    #     f.write(response)

    return response

In [297]:
# def diff_boxer(check_img_path, ref_img_path):    
#     # Load the two images
#     img1 = Image.open(check_img_path).convert("RGB")
#     img2 = Image.open(ref_img_path).convert("RGB")

#     # Compute the pixel-wise difference
#     diff = ImageChops.difference(img1, img2)

#     # Calculate bounding box of the difference
#     bbox = diff.getbbox()

#     # Make copies to draw bounding box
#     img1_bb = img1.copy()
#     img2_bb = img2.copy()

#     # Create a draw object
#     draw1 = ImageDraw.Draw(img1_bb)

#     # Draw a red bounding box around the difference area
#     if bbox:
#         draw1.rectangle(bbox, outline="red", width=5)

#     # Save the results
#     img1_bb.save(os.path.join(output_img_dir, os.path.basename(check_img_path)).replace("\\", "/"))
#     img2_bb.save(os.path.join(output_img_dir, os.path.basename(ref_img_path)).replace("\\", "/"))

#     # return (os.path.join("output/images", os.path.basename(check_img_path)).replace("\\", "/"), os.path.join("output/images", os.path.basename(ref_img_path)).replace("\\", "/"))

In [298]:
# Load HTML
with open(paths["report_path"], "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

In [299]:
scenario_dict = defaultdict(list)

with open(paths["report_path"], "r", encoding="utf-8") as f:
    lines = f.readlines()

current_scenario = None

for line in lines:
    line = line.strip()
    # Robust: check for "scenario script:" anywhere in the line
    if "<b>scenario script:" in line.lower():
        # Use regex to extract the text after :</b>
        # E.g. <b>scenario script:</b> PATH<br>
        match = re.search(r'scenario script:</b>(.*?)<br>', line, re.IGNORECASE)
        if match:
            current_scenario = match.group(1).strip()
    elif "toCheck" in line and line.lower().endswith((".png<br>", ".jpg<br>")):
        if current_scenario:
            clean_img = line.replace("<br>", "").strip()
            scenario_dict[current_scenario].append(clean_img)

# print(dict(scenario_dict))

In [300]:
# Extract mismatching image pairs
sections = soup.find_all("font", string=lambda s: s in [
    "Result and reference : simple difference",
    "Result and reference : not same size"
])

status_sections = defaultdict(list)
total_sections = len(sections)

In [301]:
status_counts_ok = 0
status_counts_nok = 0
status_counts_faulty = 0
status_counts_investigate = 0

In [302]:
sections_missing_imgs = soup.find_all("font", string=lambda s: s == "Result and reference : missing image")

for section in sections_missing_imgs:
    scenario_tag = section.find_previous(string=re.compile(r"scenario script:", re.IGNORECASE))
    scenario_script = scenario_tag.parent if scenario_tag else None

    section['color'] = "darkblue"
    status = "Faulty"
    color = "gray"
    status_counts_faulty += 1
    
    images_block = section.find_all_next("img", limit=2)

    status_tag = soup.new_tag("p")
    status_tag.string = f"Model Status: {status}"
    status_tag["style"] = f"color:{color}; font-weight:bold"
    images_block[1].insert_after(status_tag)

    sep_tag = soup.new_tag("hr")
    sep_tag["style"] = "border: 1px solid lightgray; margin: 10px 0;"
    status_tag.insert_after(sep_tag)

    section_block = [section] + [str(images_block[0]["src"])] + [str(images_block[1]["src"])] + images_block + [status_tag] + [sep_tag]
    status_sections["Faulty"].append(section_block)

In [ ]:
max_iterations = 500
iteration_count = 0
shown_errors = set()
success = True

for section in sections:
    if max_iterations:
        if iteration_count >= max_iterations:
            break
        iteration_count += 1

    section['color'] = "darkblue"
    
    images_block = section.find_all_next("img", limit=2)
    if len(images_block) < 2:
        continue
    check_img_src = images_block[0]["src"]
    ref_img_src = images_block[1]["src"]

    check_img_path = os.path.join(image_dir, os.path.basename(check_img_src))
    ref_img_path = os.path.join(image_dir, os.path.basename(ref_img_src))

    #Handling the missing images
    check_img_exists = os.path.exists(check_img_path)
    ref_img_exists = os.path.exists(ref_img_path)
    if not check_img_exists or not ref_img_exists:
        # print(f"Missing image file: {check_img_path if not check_img_exists else ''} {ref_img_path if not ref_img_exists else ''}")
        status = "Faulty"
        color = "gray"
        status_counts_faulty += 1

        status_tag = soup.new_tag("p")
        status_tag.string = f"Model Status: {status}"
        status_tag["style"] = f"color:{color}; font-weight:bold"
        images_block[1].insert_after(status_tag)

        sep_tag = soup.new_tag("hr")
        sep_tag["style"] = "border: 1px solid lightgray; margin: 10px 0;"
        status_tag.insert_after(sep_tag)

        # Collect the relevant block (font + 2 imgs + status tag)
        section_block = [section] + [str(images_block[0]["src"])] + [str(images_block[1]["src"])] + images_block + [status_tag] + [sep_tag]
        status_sections[status].append(section_block)
        continue

    try:
        img_check = Image.open(check_img_path).convert("RGB")
        img_ref = Image.open(ref_img_path).convert("RGB")
        scale_mm = compute_scale_mismatch(img_ref, img_check)

        if scale_mm > 0:
            # Skip other metrics if scale mismatch
            status, color = "Faulty", "gray"
            status_counts_faulty += 1
            
            status_tag = soup.new_tag("p")
            status_tag.string = f"Model Status: {status}"
            status_tag["style"] = f"color:{color}; font-weight:bold"
            images_block[1].insert_after(status_tag)

            sep_tag = soup.new_tag("hr")
            sep_tag["style"] = "border: 1px solid lightgray; margin: 10px 0;"
            status_tag.insert_after(sep_tag)

            # Collect the relevant block (font + 2 imgs + status tag)
            section_block = [section] + [str(images_block[0]["src"])] + [str(images_block[1]["src"])] + images_block + [status_tag] + [sep_tag]
            status_sections[status].append(section_block)
        else:
            status, color = None, None
            element_shift_info = {}
            ref2 = img_ref
            check2 = img_check
            # if img_ref.width >= 224 and img_check.height >= 224:
            #     element_shift_info = detect_element_shifts(
            #         check_img_path,
            #         ref_img_path,
            #     )
            if max(img_ref.width, img_check.height) < 224:
                if max(img_ref.width, img_check.height) <= 64:
                    img_check = realesrgan_enlarger_enhancer(img_check)
                    img_ref = realesrgan_enlarger_enhancer(img_ref)
                img_check = realesrgan_enlarger_enhancer(img_check)
                img_ref = realesrgan_enlarger_enhancer(img_ref)
            red_pixels, diff_image, has_diff, ref_white, check_white, diff_type, cluster_count = highlight_differences(img_check, img_ref, highlight_color=(255, 0, 0), threshold=30)
            diff_image.save(os.path.join(output_img_dir, os.path.basename(check_img_src)).replace("\\", "/"))
            
            response = None 
                       
            if diff_type =="global_shift":
                if ((red_pixels/(img_ref.width * img_check.height) > 0.15) and cluster_count >= 30) or ((red_pixels/(img_ref.width * img_check.height) > 0.07) and cluster_count >= 100):
                    response = "Local Model Review: Very high mismatch. The difference image likely has most or all of the content in red (please check the difference image to visually confirm). Deemed unfit for AI review."
                    status, color = "NOK", "red"
                    status_counts_nok += 1

            if not response:
                response = str(AI_analyzer(img_check, img_ref))

            if ('Verdict: Same images' in response) or ('Verdict: Similar images with insignificant differences' in response):
                status, color = "OK", "green"
                status_counts_ok += 1 
            if has_diff and status == "OK":   
                if (diff_type =="localized" and min(ref2.width, check2.height) >= 112) or (min(ref_white, check_white) == 0 and max(ref_white, check_white) >= 30):
                    status_counts_ok -= 1
                    response += "<br><i>Note: A local model has indicated a few mismatches in this pair, so it is labeled as 'Investigate.'</i><br><i>Please check the difference image to visually confirm.</i>"
                    status, color = "Investigate", "orange"
                    status_counts_investigate += 1
                    
            # if element_shift_info != {}:
            #     max_shift = element_shift_info.get("max_shift", [])
            #     std_shift = element_shift_info.get("std_shift", [])
            #     # Define tolerance in pixels
            #     tolerance = 1.0  # means std less than 1 pixel considered minimal
            #     if len(max_shift) >= 2:            
            #         if max(max_shift[0], max_shift[1]) > 6:
            #             if std_shift and all(abs(s) < tolerance for s in std_shift):
            #                 if ('Verdict: Same images' in response) or ('Verdict: Similar images with insignificant differences' in response):
            #                     if status == "Investigate":
            #                         status_counts_investigate -= 1
            #                         status, color = "OK", "green"
            #                         status_counts_ok += 1
            #             else:
            #                 if status == "OK":
            #                     status_counts_ok -= 1
            #                 status, color = "Investigate", "orange"
            #                 status_counts_investigate += 1
            #                 if ('Verdict: Same images' in response) or ('Verdict: Similar images with insignificant differences' in response) or ('Verdict: Inconclusive/Low-confidence findings; human discernment required' in response):
            #                     response += "<br><i>Note: Local model detected one or more element misalignments.</i>"
            if 'Verdict: Dissimilar/Uncomparable images' in response:
                if status == "Investigate":
                    status_counts_investigate -= 1
                status, color = "NOK", "red"
                status_counts_nok += 1            
            if status == None:
                status, color = "Investigate", "orange"
                status_counts_investigate += 1
                
            # diff_boxer(check_img_path, ref_img_path)

            high_diff_tag = soup.new_tag("p")
            high_diff_tag.string = "The image below highlights the differences between the two images above."
            high_diff_tag["style"] = "color: gray;"
            images_block[1].insert_after(high_diff_tag)

            diff_image_tag = soup.new_tag("img")
            diff_image_tag["src"] = os.path.join(os.path.basename(output_img_dir), os.path.basename(check_img_src)).replace("\\", "/")
            high_diff_tag.insert_after(diff_image_tag)

            status_tag = soup.new_tag("p")
            status_tag.string = f"Model Status: {status}"
            status_tag["style"] = f"color:{color}; font-weight:bold"
            diff_image_tag.insert_after(status_tag)

            review_tag = soup.new_tag("p")
            review_tag.append(BeautifulSoup(f"AI Review: {response}", "html.parser"))
            # review_tag["style"] = "font-style: italic" 

            sep_tag = soup.new_tag("hr")
            sep_tag["style"] = "border: 1px solid lightgray; margin: 10px 0;"
            # review_tag.insert_after(sep_tag)

            section_block = [section] + [str(images_block[0]["src"])] + [str(images_block[1]["src"])] + images_block + [high_diff_tag, diff_image_tag, status_tag, review_tag, sep_tag]
            status_sections[status].append(section_block)

    except Exception as e:
        err_msg = str(e)
        print(f"\n❌ Error comparing {check_img_src} and {ref_img_src}: {err_msg}")

        # Friendly internet connectivity message
        friendly_msg = (
            "Failed to connect to the internet. Please ensure Zscaler Service is turned off "
            "and you are connected to a private network with good connectivity. Please try again later."
        )

        # If it's an internet-related error
        if any(keyword in err_msg for keyword in ["Failed to resolve", "NameResolutionError", "Max retries exceeded"]):
            print(friendly_msg)
            shown_errors.add("internet_error")

        # If the API response was not set due to an internal code failure
        elif "name 'response' is not defined" in err_msg:
            print("Internal error: 'response' is not defined. Please ensure the API call succeeded and your internet connection is stable.")
            shown_errors.add("missing_response")

        # Handle unexpected errors (only print once per unique error)
        elif err_msg not in shown_errors:
            shown_errors.add(err_msg)

        # Clean up the output image directory before exiting
        if os.path.exists(output_img_dir):
            shutil.rmtree(output_img_dir, ignore_errors=True)

        print("\n🚫 Terminating due to a critical error.")
        success = False
        break
if success: 
    print('\n✂ Formatting the final report, please wait...')

C:\Users\z048725\AppData\Local\Temp\ipykernel_13600\690306147.py:30: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  diff_mask_img = Image.fromarray(diff_mask, mode="L")


Using Mistral API...

Difference:
 The images appear identical with no noticeable differences in elements, text, or alignment.

Verdict: Same images
Using Mistral API...

Difference:
 The images appear identical with no noticeable differences in elements, text, or alignment.

Verdict: Same images
Using Mistral API...

Difference:
 The images appear identical with no noticeable differences in elements, text, or alignment.

Verdict: Same images
Using Mistral API...

Difference:
 The images appear identical with no noticeable differences in elements, text, or alignment.

Verdict: Same images
Using Mistral API...

Difference:
 The actual image has a yellow arc on the right side which is missing in the reference image, and the positions of some icons are slightly different.

Verdict: Similar images but with one or few elements missing
Using Mistral API...

Difference:
 The images are nearly identical with no significant differences in text, icons, or alignment. The only minor difference is 

In [304]:
# Update image src in soup
for img_tag in soup.find_all("img"):
    if 'images' in str(img_tag["src"]):
        filename = os.path.basename(img_tag["src"])
        new_src = os.path.join(relative_image_path, filename).replace("\\", "/")
        # new_src = os.path.join(os.path.basename(output_img_dir), filename).replace("\\", "/")
        img_tag["src"] = new_src

In [305]:
def decide_color(status):
    if status == 'OK':
        return 'green'
    elif status == 'NOK':
        return 'red'
    elif status == 'Faulty':
        return 'grey'
    elif status == 'Investigate':
        return 'orange'
    return 'black'

In [306]:
# 1. Count status occurrences
status_counts = {
    "OK": status_counts_ok,
    "NOK": status_counts_nok,
    "Investigate": status_counts_investigate,
    "Faulty": status_counts_faulty,
}

# Remove existing duplicate summary tables if they exist
existing_tables = soup.find_all("table", attrs={"border": "1"})
for table in existing_tables:
    if "Status" in table.text and "Count" in table.text:
        table.decompose()

body = soup.body or soup

# === Create header elements ===
soup_html = BeautifulSoup(features="html.parser")

main_heading = soup_html.new_tag("h1", style="text-align:center; color:#003366; margin-bottom:5px;")
main_heading.string = variant_display_name

sub_heading = soup_html.new_tag("h2", style="text-align:center; color:#444444; margin-top:0; margin-bottom:20px;")
sub_heading.string = "AI Global Report Analysis"

summary_title = soup_html.new_tag("h3", style="text-align:center; color:#222222;")
summary_title.string = "Summary Table"

# === Create summary table HTML ===
summary_table = soup_html.new_tag("table", border="1", style="margin-left:auto; margin-right:auto; margin-bottom:20px; border-collapse:collapse;")
header = soup_html.new_tag("tr")

th_status = soup_html.new_tag("th")
th_status.string = "Status"
header.append(th_status)

th_count = soup_html.new_tag("th")
th_count.string = "Count"
header.append(th_count)

summary_table.append(header)

for status, count in status_counts.items():
    row = soup_html.new_tag("tr")
    color = {
        "OK": "green",
        "NOK": "red",
        "Investigate": "orange",
        "Faulty": "gray"
    }[status]

    cell_status = soup_html.new_tag("td", style=f"color:{color}; font-weight:bold; padding:4px 10px;")
    cell_status.string = status

    cell_count = soup_html.new_tag("td", style="padding:4px 10px;")
    cell_count.string = str(count)

    row.append(cell_status)
    row.append(cell_count)
    summary_table.append(row)

# === Extract global generation date ===
def extract_global_generation_date(soup):
    b_tag = soup.find("b", string=re.compile(r"^generation date:$", re.IGNORECASE))
    if b_tag and b_tag.next_sibling:
        date_text = b_tag.next_sibling.strip()
        if re.match(r"\d{2}-[A-Za-z]{3}-\d{4}", date_text):
            return date_text
    return "NA"

global_date = extract_global_generation_date(soup)

# Today's date
today_str = datetime.today().strftime("%d-%b-%Y")

# Date block
date_info_div = soup_html.new_tag("div")

if global_date != "NA":
    global_date_p = soup_html.new_tag("p")
    global_date_p.string = f"Global test report generation date: {global_date}"
    date_info_div.append(global_date_p)

ai_date_p = soup_html.new_tag("p")
ai_date_p.string = f"AI analysis report generation date: {today_str}"
date_info_div.append(ai_date_p)

# === Fix status explanation (remove duplicates) ===
explanation_div = soup_html.new_tag("div", style="margin-bottom:20px;")
explanation_title = soup_html.new_tag("h3")
explanation_title.string = "Status Legend"
explanation_div.append(explanation_title)

status_explanations = {
    "OK": "The images are visually and semantically aligned.",
    "Investigate": "Some mismatches found; human review advised.",
    "NOK": "Clear and significant mismatch between images.",
    "Faulty": "Image comparison skipped due to missing or size mismatch."
}

explanation_list = soup_html.new_tag("ul")
for status, explanation in status_explanations.items():
    li = soup_html.new_tag("li")
    li.string = f"{status}: {explanation}"
    explanation_list.append(li)

explanation_div.append(explanation_list)


# Report marker
report_start = soup_html.new_tag("h3", style="text-align:left; margin-top:30px;")
report_start.string = "Report:"

# === Insert all top content at once ===
insert_items = [
    main_heading,
    sub_heading,
    summary_title,
    summary_table,
    explanation_div,
    report_start,
    date_info_div,
]

for item in reversed(insert_items):  # Insert in reverse to maintain top-to-bottom order
    if isinstance(body.contents[0], Tag):
        body.insert(0, item)
    else:
        body.insert(0, item)

In [307]:
# Save results
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

In [308]:
# Load the annotated HTML
with open(output_html, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# === Step 1: Remove duplicate model status lines ===
status_tags = soup.find_all("p", string=lambda s: s and s.strip().startswith("Model Status:"))
deductions = {"OK": 0, "NOK": 0, "Investigate": 0, "Faulty": 0}

i = 0
while i < len(status_tags) - 1:
    current = status_tags[i]
    next_tag = status_tags[i + 1]
    if (current.find_next_sibling() == next_tag and
        current.string.strip() == next_tag.string.strip() and
        current.string.strip().startswith("Model Status:")):

        status_text = current.string.strip().split(":")[-1].strip()
        if status_text in deductions:
            deductions[status_text] += 1
        next_tag.decompose()
        status_tags.pop(i + 1)
    else:
        i += 1

# === Step 2: Remove duplicate headers ===
def remove_duplicate_tag(tag_name, match_text):
    """Remove all but the first tag that matches given text."""
    tags = soup.find_all(tag_name, string=lambda s: s and s.strip() == match_text.strip())
    for dup_tag in tags[1:]:
        dup_tag.decompose()

remove_duplicate_tag("h1", soup.find("h1").text if soup.find("h1") else "")
remove_duplicate_tag("h2", "AI Global Report Analysis")
remove_duplicate_tag("h3", "Summary Table")
remove_duplicate_tag("h3", "Status Legend")
remove_duplicate_tag("h3", "Report:")

# Remove duplicate summary tables (keep only first)
summary_tables = soup.find_all("table", attrs={"border": "1"})
if len(summary_tables) > 1:
    for table in summary_tables[1:]:
        table.decompose()

# Remove duplicate <ul> lists under status legend
status_lists = soup.find_all("ul")
if len(status_lists) > 1:
    for ul in status_lists[1:]:
        ul.decompose()

# === Step 3: Adjust counts in the summary table ===
# summary_table = soup.find("table", attrs={"border": "1"})
# if summary_table:
#     rows = summary_table.find_all("tr")[1:]  # Skip header
#     for row in rows:
#         status_cell, count_cell = row.find_all("td")
#         status = status_cell.text.strip()
#         if status in deductions and deductions[status] > 0:
#             original_count = int(count_cell.text.strip())
#             new_count = max(0, original_count - deductions[status])
#             count_cell.string = str(new_count)

# Save updated HTML
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

# print("✅ Duplicate Model Status entries and headers removed; summary adjusted.")

In [309]:
# Load the annotated HTML
with open(output_html, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# --- Step 1: Delete everything from body except the summary table ---
body = soup.body or soup
summary_table = body.find("table", attrs={"border": "1"})

# Preserve: h1, h2, h3 (Summary Table), table, div (explanation), h3 (Report:)
def should_preserve(tag):
    if not isinstance(tag, Tag):
        return False
    if tag.name == "h1":
        return True
    if tag.name == "h2":
        return True
    if tag.name == "h3" and tag.text.strip().lower() == "summary table":
        return True
    if tag.name == "table" and tag.has_attr("border") and tag["border"] == "1":
        return True
    if tag.name == "div" and "OK" in tag.text and "Investigate" in tag.text:
        return True
    if tag.name == "h3" and tag.text.strip().lower() == "report:":
        return True
    if tag.name == "div" and "generation date" in tag.text:
        return True
    return False

# Remove everything else
for tag in list(body.contents):
    if not should_preserve(tag):
        tag.extract()

# --- Step 2: Make summary table status names clickable using anchor links ---
for row in summary_table.find_all("tr")[1:]:  # skip header
    status_cell = row.find_all("td")[0]
    status_text = status_cell.text.strip()
    color_text = decide_color(status_text)
    anchor = soup.new_tag("a", href=f"#{status_text}", style=f"color:{color_text};")
    anchor.string = status_text
    status_cell.clear()
    status_cell.append(anchor)

# --- Step 3: Insert grouped blocks section-wise ---
for status, blocks in status_sections.items():
    # Anchor for scroll target
    anchor_tag = soup.new_tag("a", attrs={"name": status})
    body.append(anchor_tag)

    # Section heading    
    color = decide_color(status)

    header_tag = soup.new_tag("h2", style=f"color:{color}; margin-top:30px;")
    header_tag.string = f"{status} Image Comparisons"
    body.append(header_tag)

    # Add all blocks under this status
    for block in blocks:
        for element in block:
            if isinstance(element, Tag):
                body.append(element)
            else:
                # Assume it's an image src string (like "check.png"), wrap in <p>
                img_note = soup.new_tag("p", style="color:gray; font-style:italic")
                img_note.string = f"(Note: Image src reference - {element})"
                body.append(img_note)


# Save updated HTML
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

In [310]:
# Load output HTML
with open(output_html, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Find all toCheck <img> tags
for img_tag in soup.find_all("img"):
    src = img_tag.get("src", "")
    if "images" not in src or "toCheck" not in src:
        continue

    # Find which scenario this image belongs to
    matched_scenario = None
    for scenario, images in scenario_dict.items():
        for img in images:
            img_clean = img.strip().split("\\")[-1]  # images\toCheck_XXX.png → toCheck_XXX.png
            src_clean = src.replace("../input/", "").split("/")[-1]  # handle relative path
            if img_clean in src_clean:
                matched_scenario = scenario
                break
        if matched_scenario:
            break

    if matched_scenario:
        # Insert scenario <p> above this <img>
        scenario_p = soup.new_tag("p")

        # Create the <b> tag for the prefix
        b_tag = soup.new_tag("b")
        b_tag.string = "Scenario Script: "
        b_tag['style'] = "color:#003366;"  # style the bold prefix

        # Append <b> and the rest of the text
        scenario_p.append(b_tag)
        scenario_p.append(matched_scenario)

        # Insert it
        img_tag.insert_before(scenario_p)

# Save updated output
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

# print("✅ Scenario tags inserted above matching toCheck images.")


In [311]:
# ----------- STEP 1: Extract KO Segments from globalTestReport -----------

ko_scenarios = defaultdict(list)

with open(paths["report_path"], "r", encoding="utf-8") as f:
    lines = f.readlines()

current_script = None
collect_ko_lines = []
inside_scenario = False

for line in lines:
    line = line.strip()

    # New scenario starts
    if line.lower().startswith("<b>scenario script:"):
        if current_script and collect_ko_lines:
            ko_scenarios[current_script].extend(collect_ko_lines)
        current_script = line.split(":", 1)[1].strip().replace("</b>", "").strip("<br>")
        collect_ko_lines = []
        inside_scenario = True

    elif "effective value" in line:
        collect_ko_lines.append(line)

# Catch last scenario
if current_script and collect_ko_lines:
    ko_scenarios[current_script].extend(collect_ko_lines)


# ----------- STEP 2: Append KO Segments to existing output_html -----------

with open(output_html, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Add KO section at the bottom
ko_section = soup.new_tag("div")
ko_section.append(soup.new_tag("h2"))
ko_section.h2.string = "Scenario Segments with KO Checks"


for script, ko_lines in ko_scenarios.items():
    # Add each KO line first
    for ko_line in ko_lines:
        soup_line = BeautifulSoup(ko_line, "html.parser")
        clean_text = soup_line.get_text().strip()

        p_ko = soup.new_tag("p")
        p_ko.string = clean_text
        p_ko["style"] = "color:red; font-family:monospace;"
        ko_section.append(p_ko)

    # Then the Scenario Script after KO lines
    p_script = soup.new_tag("p")
    b_tag = soup.new_tag("b")
    b_tag.string = "Scenario Script: "
    p_script.append(b_tag)
    p_script.append(script)
    ko_section.append(p_script)

    sep_tag = soup.new_tag("hr")
    sep_tag["style"] = "border: 1px solid lightgray; margin: 10px 0;"
    ko_section.append(sep_tag)

br_tag1 = soup.new_tag("br")
ko_section.append(br_tag1)
br_tag2 = soup.new_tag("br")
ko_section.append(br_tag2)
br_tag3 = soup.new_tag("br")
ko_section.append(br_tag3)
br_tag4 = soup.new_tag("br")
ko_section.append(br_tag4)

# Safe append
if soup.body:
    soup.body.append(ko_section)
else:
    body_tag = soup.new_tag("body")
    body_tag.append(ko_section)
    soup.append(body_tag)

# Write back updated HTML
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

print("\n✅ Processing complete. You may now access the output folder.")
# input("Press Enter to exit...")
# sys.exit(1)


✅ Processing complete. You may now access the output folder.


In [312]:
# Read original HTML 
with open(output_html, "r", encoding="utf-8") as f: 
    soup = BeautifulSoup(f, "html.parser")

In [313]:
style_block = soup.new_tag("style")
style_block.string = """
/* Ensure images fit within the page */
img {
    max-width: 100% !important;
    height: auto !important;
    display: block;      /* prevents inline overflow issues */
    margin: 5px 0;
}

/* Prevent long text from overflowing */
p, div, span, td {
    word-wrap: break-word;
    overflow-wrap: break-word;
    white-space: normal;  /* allows wrapping instead of clipping */
}
"""
soup.body.insert(0, style_block)


[<style>
 /* Ensure images fit within the page */
 img {
     max-width: 100% !important;
     height: auto !important;
     display: block;      /* prevents inline overflow issues */
     margin: 5px 0;
 }
 
 /* Prevent long text from overflowing */
 p, div, span, td {
     word-wrap: break-word;
     overflow-wrap: break-word;
     white-space: normal;  /* allows wrapping instead of clipping */
 }
 </style>]

In [314]:
#Adding KO Checks link to the summary table
summary_table = soup.find("table")
if summary_table:
    new_row = soup.new_tag("tr")
    td1 = soup.new_tag("td", 
        style="color:black; font-weight:bold; padding:4px 10px;")
    link = soup.new_tag("a", href="#KOChecks", 
        style="color:black; text-decoration:none;")
    link.string = "KO Checks"
    td1.append(link)
    td2 = soup.new_tag("td", style="padding:4px 10px;")
    td1['nowrap'] = ""  # optional, keeps text intact
    new_row.append(td1)
    new_row.append(td2)
    summary_table.append(new_row)

# ---- 2. Add anchor before KO Checks section ----
ko_heading = soup.find("h2", string=lambda t: t and "Scenario Segments with KO Checks" in t)
if ko_heading:
    anchor = soup.new_tag("a", attrs={"name": "KOChecks"})
    ko_heading.insert_before(anchor)

In [315]:
# Inject save buttons & JS at the top (just after <body>)
buttons_html = r"""
<style>
    /* Ribbon bar at bottom */
    .bottom-ribbon {
        position: fixed;
        bottom: 0;
        left: 0;
        width: 100%;
        background-color: #01082D;
        padding: 14px;
        display: flex;              /* Make children align in one row */
        justify-content: space-between; /* Push text left, buttons right */
        align-items: center;        /* Vertical centering */
        z-index: 9998;
    }
    /* Buttons */
    .fixed-download-btn, .fixed-save-btn {
        padding: 8px 14px;
        border: none;
        border-radius: 6px;
        cursor: pointer;
        font-size: 14px;
        color: white;
        margin-left: 10px; /* spacing between buttons */
    }
    .fixed-download-btn {
        background-color: #4CAF50;
    }
    .fixed-download-btn:hover {
        background-color: #45a049;
    }
    .fixed-save-btn {
        background-color: #2196F3;
    }
    .fixed-save-btn:hover {
        background-color: #0b7dda;
    }
</style>

<div class="bottom-ribbon">
    <span style="color: white; font-size: 16px;">
        Generated by AI Image Compare tool made within RNTBCI
    </span>
    <div>
        <button class="fixed-save-btn" onclick="saveFeedback()">Save Feedback</button>
        <button class="fixed-download-btn" style="margin-right: 30px;" onclick="downloadPDF()">Download PDF</button>
    </div>
</div>

<style>
    @media print {
    /* Hide ribbon + marquee only in PDF */
    marquee, .bottom-ribbon { display: none !important; }

    /* Make images always fit page width */
    img {
        max-width: 100% !important;
        height: auto !important;
    }

    /* Long text wraps automatically */
    body {
        word-wrap: break-word;
        -webkit-print-color-adjust: exact; /* Preserve colors */
    }
}
</style>

<script src="https://cdnjs.cloudflare.com/ajax/libs/xlsx/0.18.5/xlsx.full.min.js"></script>

<!-- Store feedback JSON -->
<script id="feedback-data" type="application/json">[]</script>

<script>
    let fileHandle = null;  // Will store the chosen file handle after first selection

    async function saveFeedback() {
        try {
            let data = [];

            document.querySelectorAll(".user-status").forEach(sel => {
                let ref = sel.dataset.ref || "";
                let check = sel.dataset.check || "";
                let predicted = sel.dataset.predicted || "";
                let userStatus = sel.value || "";

                let commentEl = document.querySelector(
                    `.user-comment[data-ref="${ref}"][data-check="${check}"]`
                );
                let comment = commentEl ? commentEl.value.trim() : "";

                data.push({ref, check, predicted, userStatus, comment});

                // Sync DOM for persistence
                sel.querySelectorAll("option").forEach(opt => {
                    opt.removeAttribute("selected");
                    if (opt.value === userStatus) {
                        opt.setAttribute("selected", "selected");
                    }
                });
                if (commentEl) {
                    commentEl.textContent = comment;
                }
            });

            // Update hidden <script> with JSON
            let scriptTag = document.getElementById("feedback-data");
            scriptTag.textContent = JSON.stringify(data, null, 2).replace(/<\/script/g, "<\\/script");

            // Serialize current DOM to string
            let fullHtml = "<!DOCTYPE html>\n" + document.documentElement.outerHTML;

            // 🔹 Extract filename from current URL (works for file:// or http://)
            let currentUrl = decodeURIComponent(window.location.href);
            let fileName = currentUrl.substring(currentUrl.lastIndexOf('/') + 1) || "annotated_feedback.html";

            // 🔹 First time → ask user to choose the same HTML file
            if (!fileHandle) {
                fileHandle = await window.showSaveFilePicker({
                    suggestedName: fileName,
                    types: [{
                        description: "HTML Report",
                        accept: {"text/html": [".html"]},
                    }],
                });
            }


            // 🔹 Write content to file
            const writable = await fileHandle.createWritable();
            await writable.write(fullHtml);
            await writable.close();

            alert("Feedback saved to the same file successfully!");
        } catch (err) {
            console.error("Save failed:", err);
            alert("Failed to save feedback: " + err.message);
        }
    }

    
    function downloadPDF() {
        window.print();
    }

</script>
"""

In [316]:
body_tag = soup.body
if body_tag:
    body_tag.insert(0, BeautifulSoup(buttons_html, "html.parser"))

In [317]:
# Marquee HTML
marquee_html = r"""
<div style="background-color: yellow; padding: 6px 0;">
  <marquee behavior="scroll" direction="left" scrollamount="6" style="color: red; font-weight: bold; font-size: 16px; font-family: 'Futura', 'Poppins', 'Montserrat', 'Segoe UI', Arial, sans-serif;">
    <b>Note</b>: Do not forget to save your feedback before closing this report. Please choose the "<b>output</b>" folder in the tool's directory to save the file. Click "<b>Yes</b>" when asked "<i>File already exists. Do you want to replace it?</i>" 
  </marquee>
</div>
"""

In [318]:
h1_tag = soup.find("h1")
if h1_tag:
    h1_tag.insert_before(BeautifulSoup(marquee_html, "html.parser"))

In [319]:
# Find all "Model Status:" paragraphs and insert dropdown + comment box after them
for p in soup.find_all("p"):
    if not p.text.strip().startswith("Model Status:"):
        continue

    predicted_status = p.text.split("Model Status:")[-1].strip()

    # Look for previous notes
    ref_note = p.find_previous("p", string=lambda s: s and s.startswith("(Note: Image src reference - images\\"))
    check_note = p.find_previous("p", string=lambda s: s and s.startswith("(Note: Image src reference - images") and "toCheck" in s)

    ref_img_name = ""
    check_img_name = ""
    if ref_note:
        ref_img_name = ref_note.text.split(" - ")[-1].replace("images\\", "")
    if check_note:
        check_img_name = check_note.text.split(" - ")[-1].replace("images\\", "")

    # --- Dropdown Label ---
    dropdown_label = soup.new_tag("label")
    dropdown_label.string = "User Feedback Status: "

    # --- Dropdown ---
    select_tag = soup.new_tag("select", attrs={"name": "model_status"})
    select_tag["class"] = "user-status"
    select_tag["data-ref"] = ref_img_name
    select_tag["data-check"] = check_img_name
    select_tag["data-predicted"] = predicted_status
    for option in ["OK", "NOK", "Investigate", "Faulty"]:
        option_tag = soup.new_tag("option", value=option)
        if option == predicted_status:
            option_tag["selected"] = "selected"
        option_tag.string = option
        select_tag.append(option_tag)

    # --- Feedback Label ---
    feedback_label = soup.new_tag("label")
    feedback_label.string = " Feedback: "

    # --- Textarea ---
    textarea = soup.new_tag("textarea", rows="2", cols="50", placeholder="Please type the reason for the status change")
    textarea["class"] = "user-comment"
    textarea["data-ref"] = ref_img_name
    textarea["data-check"] = check_img_name

    # --- Line breaks ---
    br1 = soup.new_tag("br")
    br2 = soup.new_tag("br")

    # Insert in correct order *after* this <p>
    p.insert_after(br2)
    p.insert_after(textarea)
    p.insert_after(feedback_label)
    p.insert_after(br1)
    p.insert_after(select_tag)
    p.insert_after(dropdown_label)




In [320]:
feedback_script = r"""
<script>
// Dynamically update Feedback Status counts
function updateFeedbackSummary() {
    // Initialize counts
    let counts = {OK: 0, NOK: 0, Investigate: 0, Faulty: 0};

    // Loop over all dropdowns and count current selections
    document.querySelectorAll(".user-status").forEach(sel => {
        let val = sel.value || "";
        if (val && counts.hasOwnProperty(val)) {
            counts[val] += 1;
        }
    });

    // Also collect predicted counts for comparison
    let predictedCounts = {OK: 0, NOK: 0, Investigate: 0, Faulty: 0};
    document.querySelectorAll(".user-status").forEach(sel => {
        let predicted = sel.dataset.predicted || "";
        if (predicted && predictedCounts.hasOwnProperty(predicted)) {
            predictedCounts[predicted] += 1;
        }
    });

    // Check if feedback == predicted for all
    let allSame = true;
    for (let key in counts) {
        if (counts[key] !== predictedCounts[key]) {
            allSame = false;
            break;
        }
    }

    // Update the Feedback Status column in summary table
    ["OK", "NOK", "Investigate", "Faulty"].forEach(status => {
        let cell = document.getElementById("feedback-" + status);
        if (cell) {
            cell.textContent = allSame ? "-" : counts[status];
        }
    });
}

// Attach event listeners to all dropdowns
document.addEventListener("DOMContentLoaded", () => {
    document.querySelectorAll(".user-status").forEach(sel => {
        sel.addEventListener("change", updateFeedbackSummary);
    });
    // Initial call
    updateFeedbackSummary();
});
</script>
"""

# --- Modify summary table: add new <th> and <td> placeholders ---
summary_table = soup.find("table")
if summary_table:
    # Add new header
    header_row = summary_table.find("tr")
    new_th = soup.new_tag("th")
    new_th.string = "Feedback Status"
    header_row.append(new_th)

    # For each status row, add a new <td> with id for JS updates
    for row in summary_table.find_all("tr")[1:5]:  # only first 4 rows (OK, NOK, Investigate, Faulty)
        status_text = row.find("a").get_text(strip=True)
        new_td = soup.new_tag("td", id=f"feedback-{status_text}")
        new_td.string = "-"  # default
        row.append(new_td)

# Inject script at end of body
soup.body.append(BeautifulSoup(feedback_script, "html.parser"))


'\n'

In [321]:
# Save modified HTML
with open(output_html, "w", encoding="utf-8") as f:
    f.write(str(soup))

print(f"✅ Modified report saved as {output_html}")

✅ Modified report saved as output\aiAnalysisV2Report_C1AREG 10i.html
